# Consecutive shift scheduling

<div class="alert alert-block alert-info">
    &#9432; The code in this notebook requires a valid Opvious API token. You may execute it from your browser <a href="https://www.opvious.io/examples/retro/notebooks/?path=sudoku.ipynb">here</a> if you update the client's creation below to use an explicit API token.
</div>

In [1]:
%pip install 'opvious>=0.16.8'

## Formulation

In [2]:
import opvious.modeling as om

class ConsecutiveShiftScheduling(om.Model):
    """MIP model to match employees to shifts"""
    
    employees = om.Dimension()
    shifts = om.Dimension()
    level = om.Parameter.natural(employees)  # Employee level
    resource = om.Parameter.natural(shifts)  # Minimum number of employees per shift
    horizon = om.Parameter.natural()  # Number of days to schedule
    days = om.interval(1, horizon(), name="D")
    schedule = om.Variable.indicator(days, employees, shifts, qualifiers=["days"])

    @om.objective
    def maximize_total_level(self):
        return om.total(
            self.level(e) * self.schedule(d, e, s)
            for d, e, s in self.days * self.employees * self.shifts
        )
    
    @om.constraint
    def at_most_one_shift(self):
        """Each employee works at most one shift per day"""
        for d, e in self.days * self.employees:
            yield om.total(self.schedule(d, e, s) <= 1 for s in self.shifts)
            
    @om.constraint
    def enough_resource(self):
        """We have enough employees for each shift"""
        for d, s in self.days * self.shifts:
            yield om.total(self.schedule(d, e, s) >= self.resource(s) for e in self.employees)
            
    @om.constraint
    def same_consecutive_shift(self):
        """Employees keep the same shift on consecutive work days"""
        for d, e, s in self.days * self.employees * self.shifts:
            if d < self.horizon():
                yield self.schedule(d, e, s) + om.total(self.schedule(d+1, e, t) for t in self.shifts if t != s) <= 1

    @om.constraint
    def rotating_shift(self):
        """Employees rotate shifts at least once every two weeks"""
        for d, e, s in self.days * self.employees * self.shifts:
            if d < self.horizon() - 13:
                yield self.schedule(d, e, s) + self.schedule(d+7, e, s) + self.schedule(d+14, e, s) <= 2

    @om.alias(r"\lambda", days, employees)
    def unscheduled(self, d, e):
        """Convenience expression indicating whether an employee is off on a given day"""
        return 1 - om.total(self.schedule(d, e, s) for s in self.shifts)
                
    @om.constraint
    def at_most_five_shifts_per_week(self):
        """Each employee works at most 5 days per week"""
        for d, e in self.days * self.employees:
            if d < self.horizon() - 5:
                yield om.total(self.unscheduled(f, e) for f in om.interval(d, d+6)) >= 2
                
    @om.constraint
    def consecutive_time_off(self):
        """Employees have at least two days off at a time"""
        for d, e in self.days * self.employees:
            if d < self.horizon() - 1:
                yield self.unscheduled(d, e) - self.unscheduled(d+1, e) + self.unscheduled(d+2,e) >= 0

                
model = ConsecutiveShiftScheduling()
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">ConsecutiveShiftScheduling</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{employees}&: E \\
  \S^d_\mathrm{shifts}&: S \\
  \S^p_\mathrm{level}&: l \in \mathbb{N}^{E} \\
  \S^p_\mathrm{resource}&: r \in \mathbb{N}^{S} \\
  \S^p_\mathrm{horizon}&: h \in \mathbb{N} \\
  \S^a&: D \doteq \{ 1 \ldots h \} \\
  \S^v_\mathrm{schedule[days]}&: \sigma \in \{0, 1\}^{D \times E \times S} \\
  \S^o_\mathrm{maximizeTotalLevel}&: \max \sum_{d \in D, e \in E, s \in S} l_{e} \sigma_{d,e,s} \\
  \S^c_\mathrm{atMostOneShift}&: \forall d \in D, e \in E, \sum_{s \in S} \sigma_{d,e,s} \leq 1 \\
  \S^c_\mathrm{enoughResource}&: \forall d \in D, s \in S, \sum_{e \in E} \sigma_{d,e,s} \geq r_{s} \\
  \S^c_\mathrm{sameConsecutiveShift}&: \forall d \in D, e \in E, s \in S \mid d < h, \sigma_{d,e,s} + \sum_{s' \in S \mid s' \neq s} \sigma_{d + 1,e,s'} \leq 1 \\
  \S^c_\mathrm{rotatingShift}&: \forall d \in D, e \in E, s \in S \mid d < h - 13, \sigma_{d,e,s} + \sigma_{d + 7,e,s} + \sigma_{d + 14,e,s} \leq 2 \\
  \S^c_\mathrm{atMostFiveShiftsPerWeek}&: \forall d \in D, e \in E \mid d < h - 5, \sum_{x \in \{ d \ldots d + 6 \}} \lambda_{x,e} \geq 2 \\
  \S^c_\mathrm{consecutiveTimeOff}&: \forall d \in D, e \in E \mid d < h - 1, \lambda_{d,e} - \lambda_{d + 1,e} + \lambda_{d + 2,e} \geq 0 \\
  \S^a&: \forall d \in D, e \in E, \lambda_{d,e} \doteq 1 - \sum_{s \in S} \sigma_{d,e,s} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

In [3]:
import logging
import opvious

logging.basicConfig(level=logging.INFO)

client = opvious.Client.default()  # Add a token argument here to run this from the browser

# Store the formulation on the server to be able to queue a solve below
specification = await client.register_specification(model.specification(), "consecutive-shift-scheduling")

# Queue a solve attempt
problem = opvious.Problem(
    specification,
    parameters={
        "horizon": 21,
        "resource": {"open": 3, "close": 2},
        "level": {chr(65+i): i for i in range(7)}, # A, B, C, ...
    },
)
attempt = await client.queue(problem)

# Wait for the solve to complete
await client.wait_for_attempt_outcome(attempt)
outputs = await client.fetch_attempt_outputs(attempt)

INFO:opvious.client.handlers:Validated inputs. [parameters=10]
INFO:opvious.client.handlers:Attempt is queued... [elapsed=155 milliseconds]
INFO:opvious.client.handlers:Attempt is running... [elapsed=970 milliseconds, gap=inf, cuts=0, iterations=588]
INFO:opvious.client.handlers:Attempt is running... [elapsed=a second, gap=inf, cuts=0, iterations=588]
INFO:opvious.client.handlers:Attempt is running... [elapsed=3 seconds, gap=inf, cuts=0, iterations=588]
INFO:opvious.client.handlers:Attempt is running... [elapsed=4 seconds, gap=inf, cuts=0, iterations=588]
INFO:opvious.client.handlers:Attempt is running... [elapsed=7 seconds, gap=inf, cuts=0, iterations=588]
INFO:opvious.client.handlers:Attempt completed with status OPTIMAL. [objective=315.00000000000006]


In [4]:
schedule = outputs.variable("schedule")
schedule.reset_index().pivot(index=["days"], columns=["employees"], values=["shifts"]).fillna("")